# Bayesian Analysis of Anscombe's Quartet

This notebook demonstrates fitting linear models using PyMC to each of the four components of Anscombe's quartet dataset.

In [ ]:
# Import required libraries
import seaborn as sns
import pymc as pm
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pytensor

pytensor.config.cxx = ''

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")
print(f"PyMC version: {pm.__version__}")
print(f"ArviZ version: {az.__version__}")
print(f"Seaborn version: {sns.__version__}")

In [ ]:
# Load Anscombe's quartet from Seaborn public datasets
anscombe = sns.load_dataset('anscombe')
print("Anscombe's quartet dataset loaded successfully!")
print(f"Dataset shape: {anscombe.shape}")
print("\nFirst few rows:")
print(anscombe.head())
print("\nDataset info:")
print(anscombe.info())
print("\nUnique datasets:")
print(anscombe['dataset'].unique())

In [ ]:
# Visualize Anscombe's quartet
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

datasets = ['I', 'II', 'III', 'IV']
for i, dataset in enumerate(datasets):
    data_subset = anscombe[anscombe['dataset'] == dataset]
    axes[i].scatter(data_subset['x'], data_subset['y'], alpha=0.7)
    axes[i].set_title(f'Dataset {dataset}')
    axes[i].set_xlabel('x')
    axes[i].set_ylabel('y')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Function to fit linear model using PyMC
def fit_linear_model(x, y, dataset_name):
    """
    Fit a linear model equivalent to ordinary least squares using PyMC.
    The model assumes normal residuals and linear dependence on x with a constant.
    
    Model: y = alpha + beta * x + epsilon
    where epsilon ~ Normal(0, sigma)
    """
    print(f"\n{'='*50}")
    print(f"Fitting model for Dataset {dataset_name}")
    print(f"{'='*50}")
    
    with pm.Model() as model:
        # Priors for the parameters
        alpha = pm.Normal('alpha', mu=0, sigma=10)  # intercept
        beta = pm.Normal('beta', mu=0, sigma=10)    # slope
        sigma = pm.HalfNormal('sigma', sigma=1)     # residual standard deviation
        
        # Expected value of outcome
        mu = alpha + beta * x
        
        # Likelihood (sampling distribution) of observations
        y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)
        
        # Sample from the posterior
        trace = pm.sample(2000, tune=1000, return_inferencedata=True, random_seed=42, nuts_sampler='nutpie')
    
    return model, trace

In [ ]:
# Fit linear models for each of the four components of Anscombe's quartet
models = {}
traces = {}

datasets = ['I', 'II', 'III', 'IV']

for dataset in datasets:
    # Extract data for current dataset
    data_subset = anscombe[anscombe['dataset'] == dataset]
    x_data = data_subset['x'].values
    y_data = data_subset['y'].values
    
    # Fit the model
    model, trace = fit_linear_model(x_data, y_data, dataset)
    
    # Store results
    models[dataset] = model
    traces[dataset] = trace

print("\nAll models fitted successfully!")

In [ ]:
# Display summary for each of the four models
dataset = 'I'
print(f"\n{'='*60}")
print(f"MODEL SUMMARY FOR DATASET {dataset}")
print(f"{'='*60}")

# Display ArviZ summary
summary = az.summary(traces[dataset], round_to=4)
display(summary)

# Extract posterior means for interpretation
alpha_mean = traces[dataset].posterior['alpha'].mean().values
beta_mean = traces[dataset].posterior['beta'].mean().values
sigma_mean = traces[dataset].posterior['sigma'].mean().values

print(f"\nModel interpretation:")
print(f"Intercept (alpha): {alpha_mean:.4f}")
print(f"Slope (beta): {beta_mean:.4f}")
print(f"Residual std (sigma): {sigma_mean:.4f}")
print(f"Equation: y = {alpha_mean:.4f} + {beta_mean:.4f} * x")

In [ ]:
dataset = 'I'
# Plot trace plots for the main parameters
az.plot_trace(traces[dataset], var_names=['alpha', 'beta', 'sigma'],
                figsize=(12, 8))
plt.suptitle(f'Trace plots for Dataset {dataset}', fontsize=16)
plt.tight_layout()
plt.show()

# Plot posterior distributions
az.plot_posterior(traces[dataset], var_names=['alpha', 'beta', 'sigma'])
plt.suptitle(f'Posterior distributions for Dataset {dataset}', fontsize=24)
plt.tight_layout()
plt.show()

In [ ]:
# Compare results across all datasets
print("COMPARISON ACROSS ALL DATASETS")
print("="*60)

comparison_data = []
for dataset in datasets:
    alpha_mean = traces[dataset].posterior['alpha'].mean().values
    beta_mean = traces[dataset].posterior['beta'].mean().values
    sigma_mean = traces[dataset].posterior['sigma'].mean().values
    
    alpha_std = traces[dataset].posterior['alpha'].std().values
    beta_std = traces[dataset].posterior['beta'].std().values
    sigma_std = traces[dataset].posterior['sigma'].std().values
    
    comparison_data.append({
        'Dataset': dataset,
        'Intercept (α)': f"{alpha_mean:.4f} ± {alpha_std:.4f}",
        'Slope (β)': f"{beta_mean:.4f} ± {beta_std:.4f}",
        'Sigma (σ)': f"{sigma_mean:.4f} ± {sigma_std:.4f}"
    })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

print("\nNote: Values shown as mean ± standard deviation")